# Batch Gradient Descent

I'm going to implement Batch Gradient Descent with early stopping for Softmax Regression without using Scikit-Learn. I'll use to Iris data set from Scikit-learn to do this.

In [1]:
from sklearn import datasets
import numpy as np
iris = datasets.load_iris()
print(iris['DESCR'])

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

In [2]:
iris.data.shape

(150, 4)

In [3]:
type(iris.data)

numpy.ndarray

In [4]:
X = iris.data[:, (2,3)]
y = iris.target

In [5]:
eta = 0.1
n_iterations = 1000
theta = np.random.randn(X.shape[0], X.shape[1])
m = 100

In [6]:
X.shape

(150, 2)

In [8]:
X_b = np.c_[np.ones((X.shape[0], 1)), X]

In [36]:
X_b.shape

(150, 3)

In [7]:
y.shape

(150,)

In [35]:
theta.shape

(150, 2)

In [37]:
prob_k

array([[9.39735790e-03, 5.19881332e-01],
       [2.13129780e-06, 1.49828871e-25],
       [9.90600511e-01, 4.80118668e-01]])

#### Softmax function

In [14]:
def prob_softmax(score):
    return np.exp(score)/sum(np.exp(score))

In [55]:
for iteration in range(n_iterations):
    prob_k = prob_softmax(X_b.T.dot(theta))
    ind = np.unravel_index(np.argmax(prob_k, axis=None), prob_k.shape)
    gradients = 2/m * X_b.T.dot(theta.T.dot(prob_k[ind[0]]) - y)
    print(gradients.shape)
    print(theta.shape)
    break
#     theta = theta - eta * gradients

(3,)
(150, 2)


In [40]:
prob_k

array([[9.39735790e-03, 5.19881332e-01],
       [2.13129780e-06, 1.49828871e-25],
       [9.90600511e-01, 4.80118668e-01]])

In [44]:
prob_k[np.unravel_index(np.argmax(prob_k, axis=None), prob_k.shape)[0]]

array([0.99060051, 0.48011867])